# Laboratory 1: setting up Twitter API

## Set up

In [ ]:
import os
os.environ['TOKEN'] = 'your token here'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import libraries

In [ ]:
import requests 
import pandas as pd 
import time
import re
import string
import ast

If you need to download a library, use the following code, just specify the name of the library you need (here we downloaded emoji library)

In [ ]:
!pip install emoji

### Set up headers

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [ ]:
headers = create_headers(os.environ['TOKEN'])

## Download data

Date format and other parameter explanations available here:

https://developer.twitter.com/en/docs/twitter-api/premium/search-api/api-reference/premium-search#DataParameters

In [ ]:
def create_url(keyword, start_date, end_date, env_label, endpoint="fullarchive"):
    
    search_url = "https://api.twitter.com/1.1/tweets/search/{}.json".format(endpoint+"/"+env_label) 

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 'fromDate': start_date, 'toDate': end_date}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    if next_token is not None and next_token != '':
      params['next'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
def get_data(keyword, start_time, end_time, next_token, env_label, endpoint):
  results = []
  dCounter = 0
  while next_token is not None and dCounter < 10:  #change dCounter to 10 to retrieve 1k data
    ##this part here for one request
    url = create_url(keyword, start_time,end_time, env_label, endpoint)
    json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
    
    if "results" in json_response:
      results.extend(json_response["results"])
    ### up until this point
    if "next" in json_response:
        next_token = json_response["next"]
        dCounter += 1
    else:
      next_token = None
    time.sleep(1)
  
  return results

In [ ]:
def get_single_response(keyword, start_time, end_time, env_label, endpoint):
  #endpoint can be fullarchive or 30day
  results = []
  url = create_url(keyword, start_time,end_time, env_label, endpoint)
  json_response = connect_to_endpoint(url[0], headers, url[1])
  
  if "results" in json_response:
    results.extend(json_response["results"])
  
  return results

In [ ]:
#if you are using the 30day endpoint, make sure you specify dates that are within 30day range!
#To get small data of size "100" use, get_single_response
#change the hashtags to your liking
# with "get_data" we are querying for 1k data - see the definition of get_data func above
tweets = get_data("(gulpanra OR JusticeforGulPanra OR StopGenocideOfTransgenders OR TransLivesMatter) lang:en", "202001010000", "202012310000", "","NSdev", "fullarchive")
#tweets = get_single_response("(EndTransViolence OR BeelaCrisis OR JusticeForBijlee OR JusticeForToffi OR JusticeforGulPanra OR TransLivesMatter) lang:en", "202001010000", "202112310000","NSdev", "fullarchive")

### Inspect data

In [ ]:
len(tweets)

In [ ]:
tweets[0]

# Laboratory 2: working with Twitter data

First, we want to convert the data into Pandas DataFrame. This format enables us easy manipulation of the data as well as saving/loading data.

Since we have our tweets saved as a list of dictionaries, we can easily convert it to DataFrame by executing the cell blow.

In [ ]:
tweets_df = pd.DataFrame(tweets)

In [ ]:
tweets_df

### Saving the results

Once we have our Tweets as a DataFrame it is a good idea to save it on the disk. 

Be mindful of the fact that the storage of a Colab notebook is deleted everytime runtime is interrupted or restarted, so you need to manually download it to your computer or mount your Google Drive and save it there (this option is unavailable if you're using university's email account for Drive).

In [ ]:
path = "Desktop\network proje" #enter the path to your Drive or leave this as default

We can save it as a comma-separated values file, which enables opening it in a spreadsheet editor and inspecting it.

In [ ]:
tweets_df.to_csv(path+"trans_case1.csv", index=False)

In order to preserve datatypes, we should save it as a parquet or pickle file.

In [ ]:
tweets_df.to_pickle(path+"trans_case1.pkl")

### Loading the data

If you want to load the results you have previously saved, simply execute the next code, specifying the path to the file.

You will need to either upload it to the Colab workspace or copy the path to the file on Drive.

In [ ]:
tweets_df = pd.read_pickle("trans_case1.pkl")

In [ ]:
tweets_df

### Preprocessing the data

In our dataframe we have the entire Tweet object. Some columns that might be of particular interest to us are: 

*   created_at - date when Tweet was posted
*   id/id_str - unique Tweet identifiers
*   text - the content of the Tweet
*   user - information about the user who posted the Tweet
*   retweeted_status  - information about the original Tweet
*   quote/reply/retweet/favorite count - Tweet metrics
*   entities - hashtags, urls, user_mentions present in Tweet

We can filter the dataframe and keep only columns we are interested in. You can pick which columns you'd like to keep and put them int the column_list below.



In [ ]:
#Everything is filtered below in Umut's code, no need to do it now

#tweets_filtered = tweets_df.copy() #it's a good idea to work on the copy of original dataframe, so we can always go back to it if we mess something up
#column_list = ["created_at", "id_str", "text", "user", "retweeted_status", "quote_count", "reply_count", "retweet_count", "favorite_count", "entities"]
#tweets_filtered = tweets_filtered[column_list]

In [ ]:
tweets_filtered

In [ ]:
def preProcessData(df):
    for index, row in df.iterrows():
        # get full_text of the tweet
        if row['truncated']:
            tweet = {"full_text": ast.literal_eval(row['extended_tweet'])['full_text']}
        else:
            # if the tweet is retweeted, this status will not be na
            if ('retweeted_status' in row) and (not pd.isna(row['retweeted_status'])):
                originalTweet = ast.literal_eval(row['retweeted_status'])
                userTagName = originalTweet['user']['screen_name']

                # if the text is truncated
                if originalTweet['truncated']:
                    tweet = {"full_text": "RT @" + userTagName + ": " + originalTweet['extended_tweet']['full_text']}
                else:
                    tweet = {"full_text": "RT @" + userTagName + ": " + originalTweet['text']}

                del originalTweet
                del userTagName

            # if the tweet is original and not truncated
            else:
                tweet = {"full_text": row['text']}

        # get Username & User Tag & Location & Verification status of account
        tweet["username"] = ast.literal_eval(row['user'])['name']
        tweet["screen_name"] = ast.literal_eval(row['user'])['screen_name']
        tweet["location"] = ast.literal_eval(row['user'])['location']
        tweet["is_verified"] = ast.literal_eval(row['user'])['verified']

        # get Quote & Fav & RT & Reply counts
        tweet["quote_count"] = row['quote_count']
        tweet["favorite_count"] = row['quote_count']
        tweet["retweet_count"] = row['retweet_count']
        tweet["reply_count"] = row['reply_count']

        # get Tweet creation date&time
        tweet["created_at"] = row['created_at']

        # get TweetID
        tweet["tweet_id"] = row['id']

        tweets.append(tweet)

        del tweet
        del row
        del index


def countNumberOfRetweetedTweets(tweets):
    # Separate RTs and original tweets
    for tweet in tweets:
        if bool(re.match(r'(RT @[\w]+:)', tweet["full_text"])):
            rts.append(tweet)
        else:
            non_rt.append(tweet)
        del tweet


def countTweetsFromVerifiedAccounts(tweets):
    # Distinguish tweets according to their account verification status    
    for tweet in tweets:
        if tweet["is_verified"]:
            tweets_from_verified_accounts.append(tweet)
        else:
            tweets_from_regular_accounts.append(tweet)

        del tweet


# filter locations according to city names given in listOfCities
def filterLocations(tweets, listOfCities):
    filtered_tweets = []

    for tweet in tweets:
        for index, city in listOfCities.iterrows():
            if city['city'] in tweet['location']:
                filtered_tweets.append(tweet)
                break
    return filtered_tweets


In [ ]:
df1 = pd.read_csv('trans_case1.csv')
df2 = pd.read_csv('trans_case_mix.csv')
df3 = pd.read_csv('trans_case3.csv')
tweets = []
tweets_from_verified_accounts = []
tweets_from_regular_accounts = []

rts = []
non_rt = []

#pakistanCities = pd.read_csv('pakistanCities.csv')
#egyptCities = pd.read_csv('egyptCities.csv')
#dfs = [df1, df2 ,df3]

# all data of 3 different cases are concataneted 
dfs = pd.concat(map(pd.read_csv, ['trans_case1.csv', 'trans_case_mix.csv','trans_case3.csv']))

#index column was not incrementing from 0 through 3000 for 3k data. So we resetted it to have an index column of 0-2999
dfs.reset_index(drop=True, inplace=True)

df_final = dfs.copy() # to work on a copy, copied the original df
preProcessData(df_final) # full text, column filtering are applied to the df
countNumberOfRetweetedTweets(tweets) # count of Retweeted Tweets
countTweetsFromVerifiedAccounts(tweets) # count of Tweets From Verified Accounts

"""
# preprocess all the given hashtags and build 
for df_instance in dfs:
    preProcessData(df_instance)

    del df_instance

countNumberOfRetweetedTweets(tweets)
countTweetsFromVerifiedAccounts(tweets)

#original_tweets_from_pakistan = filterLocations(non_rt, pakistanCities)
#all_tweets_from_pakistan = filterLocations(tweets, pakistanCities)
"""

df_final

In [ ]:
# DataFrame of whole network and in the last line, it saved it as a csv file
tweets_df = pd.DataFrame(tweets)
tweets_df
tweets_df.to_csv("trans_full_network.csv", index=False) # change the name according to your case

In [ ]:
tweets_from_verified_accounts_df = pd.DataFrame(tweets_from_verified_accounts)
tweets_from_verified_accounts_df

In [ ]:
# DataFrame of tweets from regular accounts
tweets_from_regular_accounts_df = pd.DataFrame(tweets_from_regular_accounts)
tweets_from_regular_accounts_df

In [ ]:
# retweeted tweets
len(rts) # length of the rt tweets
rts_df = pd.DataFrame(rts)
rts_df

In [ ]:
# not retweeted tweets, basically the remaining tweets
len(non_rt) # length of not rt tweets
original_tweets_df = pd.DataFrame(non_rt)
original_tweets_df

## Extracting words/hashtags

There are many ways to build networks from the data we download from Twitter.

One possibility is to have a bipartite network of Tweets and words/hashtags and then observe word, hashtag or word-hashtag projections.

### Extracting words

In order to extract words, we first need to clean the Tweet text. This way we will remove punctuation, hashtags/mentions/urls (they are preserved in the entity column anyway). We will also turn all letters to lowercase.

You can also consider removing stopwords, removing words that are not in the english language corpora, lematizing the words, etc. I suggest you research nltk library and its possibilities.

In [ ]:
import re
import string

In [ ]:
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) # remove mentions
    tweet = re.sub("#[A-Za-z0-9]+", "",tweet) # remove hashtags
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) # remove http links
    tweet = " ".join(tweet.split())
    tweet = str.lower(tweet) #to lowercase
    table = str.maketrans(dict.fromkeys(string.punctuation)) 
    tweet = tweet.translate(table)# remove punctuation         
    return tweet

In [ ]:
tweets_filtered["clean_text"] = tweets_df["full_text"].map(cleaner)
print(tweets_filtered['clean_text'])

We are going to loop through the dataframe and then through the words in the clean text. We are going to add the words as keys to dictionary and use their frequencies as values.

In [ ]:
#initialize an empty dict
unique_words = {}
for row in tweets_filtered.clean_text:
  for word in row.split(" "):
    #if the word is encountered for the first time add to dict as key and set its value to 0
    unique_words.setdefault(word,0)
    #increase the value (i.e the count) of the word by 1 every time it is encountered
    unique_words[word] += 1

In [ ]:
#remove empty word
unique_words.pop("")
#remove word 'rt'
unique_words.pop("rt")


We can inspect the words as a dataframe. 


You can always save this dataframe as .csv for future reference.

In [ ]:
#In the last line, it saves word count df as csv file. It's better to save it so that maybe we'll use it later for the presentation
uw_df = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
print(uw_df)
uw_df.rename(columns = {'index':'Word', 0:'Count'}, inplace=True)
uw_df.sort_values(by=['Count'], ascending=False, inplace=True)
uw_df
uw_df.to_csv("full_network_word_count.csv", index=False) # change the name to your liking

### Extracting the hashtags

We are going to loop through the dataframe and then through the hashtags in the entities. We are going to add the hashtags as keys to dictionary and use their frequencies as values. At the same time, we are going to save them in a list and add them to a separate column to facilitate our future work.

In [ ]:
# no need to run this cell. We already have screen_name column by this point

"""
#TASK-1

sc_name = {}
tweets_filtered["screen_name"] = ""

for idx, row in tweets_filtered.iterrows():
  screen_name_list = []
  for user_mentions in row["entities"]["user_mentions"]:
    sc_name.setdefault(user_mentions['screen_name'], 0)
    sc_name[user_mentions["screen_name"]] += 1
    screen_name_list.append(user_mentions["screen_name"])
  tweets_filtered.at[idx,"screen_names"] = screen_name_list
sc_name

sc_df = pd.DataFrame.from_dict(sc_name, orient='index').reset_index()
sc_df.rename(columns = {'index':'Username', 0:'Count'}, inplace=True)
sc_df.sort_values(by=['Count'], ascending=False, inplace=True)
sc_df
#END OF TASK-1

"""

In [ ]:
unique_hashtags = {}
#Adds hashtag column in the filtered data frame
tweets_filtered["hashtags"] = ""

for idx, row in tweets_filtered.iterrows():
  hashtag_list = []
  for hashtag in row["entities"]["hashtags"]:
    unique_hashtags.setdefault("#"+hashtag["text"], 0)
    unique_hashtags['#'+hashtag["text"]] += 1
    hashtag_list.append(hashtag["text"])
  tweets_filtered.at[idx,"hashtags"] = hashtag_list
unique_hashtags

In [ ]:
uh_df = pd.DataFrame.from_dict(unique_hashtags, orient='index').reset_index()
uh_df.rename(columns = {'index':'Hashtag', 0:'Count'}, inplace=True)
uh_df.sort_values(by=['Count'], ascending=False, inplace=True)

In [ ]:
#In the last line, it saves hashtag count df as csv file. It's better to save it so that maybe we'll use it later for the presentation
uh_df
uh_df.to_csv("full_network_hashtag_count.csv", index=False) # change the name to your liking

## Building the network

We are going to use the networkx library, which is a Python library that enables network science analysis of the data.

We are going to use it to create our network and extract edgelist from it, since we can easily import it to Gephi (a software we are going to see in visualization labs).

However, it offers implemented algorithms for analysis (for example PageRank) that you can use out-of-box to analyze your network.

But first, we will loop through our dataframe and connect words and hashtags if they appear together in the same Tweet.

In [ ]:
import itertools
import networkx as nx

In [ ]:
uh = unique_hashtags.keys()
uw = unique_words.keys()  

In [ ]:
# No need to run this cell

"""
#SELEN
sc=sc_name.keys()
sc

"""

In [ ]:
#It creates pairs from all words.
# From this cell on, we will create 3 different networks:
# Network1 = words+hashtags
#Network2 = words only
#Network3 = hashtags only
#This cell is for Network1
network = {}
network_key = 0
for index, row in tweets_filtered.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    combined_list = ['#'+hashtag for hashtag in row["hashtags"] if '#'+hashtag in unique_hashtags] + [word for word in str.split(row["clean_text"], " ") if word in uw]
    #itertool product creates Cartesian product of each element in the combined list
    for pair in itertools.product(combined_list, combined_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in network):
            network.setdefault(pair,0)
            network[pair] += 1 
network_df = pd.DataFrame.from_dict(network, orient="index")
network_df

In [ ]:
#This cell is for Network2
# NETWORK OF ONLY WORDS
networkWords = {}
networkWords_key = 0
for index, row in tweets_filtered.iterrows():
    word_list =[word for word in str.split(row["clean_text"], " ") if word in uw]
    #itertool product creates Cartesian product of each element in the word list
    for pair in itertools.product(word_list, word_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in networkWords):
            networkWords.setdefault(pair,0)
            networkWords[pair] += 1 
networkWords_df = pd.DataFrame.from_dict(networkWords, orient="index")
networkWords_df

In [ ]:
#This cell is for Network3
# NETWORK OF ONLY HASHTAGS
networkHashtags = {}
networkHashtags_key = 0
for index, row in tweets_filtered.iterrows():
    #hashtags extracted from Tweet do not have the # sign in front of them but we will add it to differentiate hashtags from words
    hashtag_list = ['#'+hashtag for hashtag in row["hashtags"] if '#'+hashtag in unique_hashtags]
    #itertool product creates Cartesian product of each element in the word list
    for pair in itertools.product(hashtag_list, hashtag_list):
        #exclude self-loops and count each pair only once because our graph is undirected and we do not take self-loops into account
        if pair[0]!=pair[1] and not(pair[::-1] in networkHashtags):
            networkHashtags.setdefault(pair,0)
            networkHashtags[pair] += 1 
networkHashtags_df = pd.DataFrame.from_dict(networkHashtags, orient="index")
networkHashtags_df

In [ ]:
#This cell is for Network1
network_df.reset_index(inplace=True)
network_df.columns = ["pair","weight"]
network_df.sort_values(by="weight",inplace=True, ascending=False)
network_df

In [ ]:
#This cell is for Network2
# FOR WORD NETWORK
networkWords_df.reset_index(inplace=True)
networkWords_df.columns = ["pair","weight"]
networkWords_df.sort_values(by="weight",inplace=True, ascending=False)
networkWords_df

In [ ]:
#This cell is for Network3
# FOR HASHTAG NETWORK
networkHashtags_df.reset_index(inplace=True)
networkHashtags_df.columns = ["pair","weight"]
networkHashtags_df.sort_values(by="weight",inplace=True, ascending=False)
networkHashtags_df

In [ ]:
#This cell is for Network1
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted = []
for edge in network:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1:
    up_weighted.append((edge[0],edge[1],network[edge]))

G = nx.Graph()
G.add_weighted_edges_from(up_weighted)


In [ ]:
#This cell is for Network2 & Network3
# list of 3-element tuples for word and hastag networks
up_weighted_words = []
for edge in networkWords:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1:
    up_weighted_words.append((edge[0],edge[1],networkWords[edge]))

G_words = nx.Graph()
G_words.add_weighted_edges_from(up_weighted_words) # words graph

#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes and w is a number representing weight
up_weighted_hashtags = []
for edge in networkHashtags:
    #we can filter edges by weight by uncommenting the next line and setting desired weight threshold
    #if(network[edge])>1:
    up_weighted_hashtags.append((edge[0],edge[1],networkHashtags[edge]))

G_hashtags = nx.Graph()
G_hashtags.add_weighted_edges_from(up_weighted_hashtags) # hashtags graph

In [ ]:
# No need to run this cell, we'll use Gephi for visualization anyways
"""
import matplotlib.pyplot as plt
nx.draw(G, with_labels=True, node_size=1.5, alpha=0.3, arrows=True)
plt.show()
nx.draw(G, with_labels=True)
plt.show()

### ploting words network
nx.draw(G_words, with_labels=True, node_size=1.5, alpha=0.3, arrows=True)
plt.show()
nx.draw(G_words, with_labels=True)
plt.show()

### ploting hashtags network
nx.draw(G_hashtags, with_labels=True, node_size=1.5, alpha=0.3, arrows=True)
plt.show()
nx.draw(G_hashtags, with_labels=True)
plt.show()

"""


In [ ]:
# This cell is for Network1
# WORDS&HASHTAGS NETWORK NODES&EDGES
print(len(G.nodes())) # nodes=each word in the texts
print(len(G.edges())) # edges=pairs for each word
#G.edges
#G.nodes

In [ ]:
# This cell is for Network2 & Network3

#WORDS NETWORK NODES&EDGES
print(len(G_words.nodes()))
print(len(G_words.edges()))
#HASHTAGS NETWORK NODES&EDGES
print(len(G_hashtags.nodes()))
print(len(G_hashtags.edges()))

In [ ]:
#In this cell, we rank the nodes using Pagerank function and print the top20 nodes with the highest pageranks

from collections import Counter

pr = nx.pagerank(G, alpha=0.9) # we ranked the nodes of Network1 with pagerank

# Let's print the top-20 pairs
c = Counter(pr)
top_20_network = c.most_common(20) 
print(top_20)
print("********************")

pr_word = nx.pagerank(G_words, alpha=0.9) # we ranked the nodes of Network2 with pagerank

# Let's print the top-20 pairs
c = Counter(pr_word)
top_20_word = c.most_common(20) 
print(top_20_word)
print("********************")


pr_hashtag = nx.pagerank(G_hashtags, alpha=0.9) # we ranked the nodes of Network3 with pagerank

# Let's print the top-20 pairs
c = Counter(pr_hashtag)
top_20_hashtag = c.most_common(20) 
print(top_20_hashtag)

#### SAVE EDGELIST

In [ ]:
# Now we're going to save edgelists and nodelists to use for Gephi later
# Change the names according to your case
filename1 = "./network_edgelist_trans.csv"
filename2 = "./word_edgelist_trans.csv"
filename3 = "./hashtag_edgelist_trans.csv"

In [ ]:
nx.write_weighted_edgelist(G, filename1, delimiter=",") # Graph name must be changed!!
nx.write_weighted_edgelist(G_words, filename2, delimiter=",") # Graph name must be changed!!
nx.write_weighted_edgelist(G_hashtags, filename3, delimiter=",") # Graph name must be changed!!

In [ ]:
# If this cell doesn't work in your device, no worries you can skip it
#add header with appropriate column names (works on collab and Linux/Mac(?))
!sed -i.bak 1i"Source,Target,Weight" ./network_edgelist_trans.csv # Graph name must be changed!!
!sed -i.bak 1i"Source,Target,Weight" ./word_edgelist_trans.csv # Graph name must be changed!!
!sed -i.bak 1i"Source,Target,Weight" ./hashtag_edgelist_trans.csv # Graph name must be changed!!

### Create Node List


In [ ]:
# This creates a csv file of nodes for Network2
word_nodes = pd.DataFrame.from_dict(unique_words,orient="index")
word_nodes.reset_index(inplace=True)
word_nodes["Label"] = word_nodes["index"]
word_nodes.rename(columns={"index":"Id",0:"delete"},inplace=True)
word_nodes = word_nodes.drop(columns=['delete'])

word_nodes
word_nodes.to_csv("word_nodelist_trans.csv",index=False)

In [ ]:
# This creates a csv file of nodes for Network3
hashtag_nodes = uh_df.copy()
hashtag_nodes["Label"] = hashtag_nodes["Hashtag"]
hashtag_nodes.rename(columns={"Hashtag":"Id"},inplace=True)
hashtag_nodes = hashtag_nodes.drop(columns=['Count'])
hashtag_nodes
hashtag_nodes.to_csv("hashtag_nodelist_trans.csv",index=False)

#### SAVE NODELIST

In [ ]:
# This joins the two nodelists above and creates a csv file of nodes for Network1
nodelist = hashtag_nodes.append(word_nodes, ignore_index=True)
nodelist
nodelist.to_csv("network_nodelist_trans.csv",index=False)

Tasks: 

*   Extract username of user who posted the tweet into a column "screen_name". Follow the procedure we used to get the hashtags.
*   Create a network of users using the mention relation. Is this a directed or undirected graph?
*   We created a network where nodes are mixed (both words and hashtags). Create network of words only and one of hashtags only.
* Pick one of these network and rank the nodes using PageRank centrality. Extract information about top-20 rated nodes.



